In [16]:
"""This file is a test playground for new scripts and functions."""

%load_ext autoreload
%autoreload 2

###### USER SETTINGS ######
VIDEO = "truck" # pick video suffix from algo_input_videos/ folder
RESOLUTION_ATT = 100 # resolution of get_attention for DINO model
MODEL = 'hybrid' # MiDaS model type ('small', 'hybrid', 'large')
THRESHOLD_VAL = 0.35 # threshold of attention+depth combination
BIAS = 0.75 # bias towards attention for attention+depth combination
SCALE = 4 # scaling of combined array (scale*[16, 9])
DISPLAY_DIMS = (4,7) # HASEL haptic display dimensions, H x W (pixels)
FRAME_SKIP = 5 # interval for how often to calculate algorithm (then interpolate between)
DEVICE = "cpu"

###### INITIALIZATIONS ######
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import matplotlib.animation as animation
import visual_haptic_utils.algo_preprocessing as algo

###### MAIN ######
## Setting up torch device and model, video input
device = torch.device(DEVICE)
cap = cv2.VideoCapture("algo_input_videos/video_" + VIDEO + ".mp4") #use video
model = algo.VisualHapticModel(device=device,
                               resolution_attention=RESOLUTION_ATT,
                               depth_model_type=MODEL,
                               threshold_value=THRESHOLD_VAL,
                               bias=BIAS,
                               scaling_factor=SCALE,
                               display_dim=DISPLAY_DIMS)

frame_num = 1
output_list = []
depth_list = []
attention_list = []

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using cache found in C:\Users\bjohnson/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\bjohnson/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\bjohnson/.cache\torch\hub\facebookresearch_dino_main


In [17]:
frame = algo.grab_video_frame(cap)

In [13]:
attention_frame = model.get_attention(frame)
depth = model.get_depth(frame)
depth_frame = model.remove_gradient(depth)

In [14]:
combined_frame = model.get_combined(depth_frame, attention_frame)

In [7]:
threshold_frame = model.get_threshold(combined_frame)
output = model.get_downsample(threshold_frame)

In [18]:
output = model.run(frame) # run the visual-haptic algorithm

In [ ]:

            if output_list:#skip first iteration
                last_output = output_list[-1]
                last_depth = depth_list[-1]
                last_att = attention_list[-1]
                for frame in range(1, FRAME_SKIP):
                    interp_vec = np.zeros(DISPLAY_DIMS)
                    interp_vec2 = np.zeros(model.depth_frame.shape)
                    interp_vec3 = np.zeros(model.attention_frame.shape)
                    for row in range(DISPLAY_DIMS[0]):
                        for col in range(DISPLAY_DIMS[1]):
                            interp_vec[row, col] = np.linspace(last_output[row, col], output[row, col], FRAME_SKIP+1)[frame]
                    output_list.append(interp_vec)

            output_list.append(output)
            print(len(output_list))

        frame_num += 1
        print(frame_num)